# Web Sraping

**Goal**: Retrieve articles from *ekathimerini.com*

To extend the use of this .ipynb for other sites we only need to change the cssSelectors depending on the layout of the Page

END RESULTS IN [GDRIVE Folder](https://drive.google.com/drive/folders/1rde9viDnmkJIRKaLTsGHufBVIMw2q3fl?usp=sharing)

## Load Drive
Load google drive for storage

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Setup
1. install selenium
2. update and install chromedriver 
3. insert chromedriver to $PATH
4. Basic python imports 
5. chromedriver options setup

In [ ]:
!pip install selenium

     |████████████████████████████████| 968 kB 8.3 MB/s 
     |████████████████████████████████| 138 kB 69.9 MB/s 
     |████████████████████████████████| 359 kB 78.0 MB/s 
     |████████████████████████████████| 3.6 MB 46.8 MB/s 
     |████████████████████████████████| 55 kB 5.2 MB/s 
     |████████████████████████████████| 58 kB 8.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 https://developer.download.nvidia.com/compute/c

In [ ]:
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [ ]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pickle

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=chrome_options)

## Generate sitemap.xml post urls
Used to parse this [page](https://www.ekathimerini.com/sitemap.xml) </br>
We generate a list of all sitemap.xml urls based on dates we are interested in. Dates of interest range from 01-01-2019 to 12-04-2022 

In [ ]:
'''
 Find all links of sitemap 
 all Articles posted in 01-2019 can be found on this sample link
 https://www.ekathimerini.com/sitemap-pt-post-2019-01.xml
'''


baseUrl = 'https://www.ekathimerini.com/sitemap-pt-post-'

dates=[]
years=[2019,2020,2021,2022]
for i in range(1,13):
  for year in years:
    if year==2022 and i>=5:
      continue
    else:
      dates.append(str(year)+'-'+str(i).zfill(2))
  
urls = [baseUrl+date+'.xml' for date in dates]
print(urls)


['https://www.ekathimerini.com/sitemap-pt-post-2019-01.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2020-01.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2021-01.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2022-01.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2019-02.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2020-02.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2021-02.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2022-02.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2019-03.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2020-03.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2021-03.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2022-03.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2019-04.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2020-04.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2021-04.xml', 'https://www.ekathimerini.com/sitemap-pt-post-2022-04.xml', 'https://www.ekathimerini.com/sitemap-p

## Iterate through all post-xml pages
We use the urls generated above. For every url we extract all the article urls contained in that particular post-xml url. All articles are then stored in  [google drive folder](https://drive.google.com/drive/folders/1rde9viDnmkJIRKaLTsGHufBVIMw2q3fl?usp=sharing)

In [ ]:

allUrls=[]

for index,url in enumerate(urls):
  driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
  driver.get(url)
  total=driver.find_elements(By.TAG_NAME,"a")
  urlOI=[]
  for idx, val in enumerate(total):
    if 'kathimerini' in val.text:
      urlOI.append(val.text)
    else:
      continue
  
  '''
  #Run once
  with open("./drive/MyDrive/webScraping/kathimerini/links/file"+dates[index]+".txt", "w") as output:
    for item in urlOI:
        output.write("%s\n" % item)
  '''

## Parse Links of Interest and Extract corpus

Now we iterate through every saved .txt file (which contains links based on date of publishment). We only get **100** links per month from 01-2019 up until 04-2022. In total we have 100*40 = 4000 articles (approx. runtime to create that .csv is ~180-200 minutes in google colab)

In [ ]:
# Used for .csv writing (csv writer treated strings as iterables and I had to find a workaround)
class non_iter_str(str):
    def __iter__(self):
        yield self

In [ ]:
import csv
import numpy as np


#Create the header for .csv file
#Remove X in order to rerun
with open("X./drive/MyDrive/webScraping/kathimerini/corpus.csv", "w") as file:
        header=[non_iter_str('url'),non_iter_str('date'),non_iter_str('title'),non_iter_str('corpus')]
        write = csv.writer(file)
        write.writerows([header])


#Iterate through every month
for i in range(len(dates)):
  with open("./drive/MyDrive/webScraping/kathimerini/links/file"+dates[i]+".txt", "r") as input:
    
  
    print("Currently at ",dates[i])

    # Prevent some issues regarding chromedriver sync
    sleep(0.5) 
    
    # Read all articles in .txt file
    inpt = input.readlines()  
    

    # Get 100 evenly spaced articles  
    indices = np.round(np.linspace(0,len(inpt)-1,100)).astype(int)
    print(indices)
    
    for i in range(len(indices)):
      
      #URL of interest
      tempUrl = inpt[indices[i]]
    
      
      # Extract article id from url (later used in css selector)
      idArticle=str([int(s) for s in tempUrl.split("/") if s.isdigit()][0])
      

      #Initiliaze driver
      driver = webdriver.Chrome('chromedriver',options=chrome_options)
      driver.get(tempUrl)
     
      
      
      #Get Corpus (cssSelector is found manually if we inspect a page but remains the same for every article)
      cssSelector = '#post-'+str(idArticle)+' > div.entry-content'
      #We use try because there are a few (insignificant) articles that have a peculiar css layout 
      try:
        corpusElement= driver.find_element(by=By.CSS_SELECTOR, value=cssSelector)
      except:
        continue
      
      
      #Get Date
      dateCssSelector = '#post-'+str(idArticle)+'> div.entry-meta > div > span > time.meta-date-modified.updated'
      dateElement = driver.find_element(by=By.CSS_SELECTOR, value=dateCssSelector) 
      
      #Get title
      titleCssSelector = '#post-'+str(idArticle)+' > header > h1 '
      titleElement = driver.find_element(by=By.CSS_SELECTOR, value=titleCssSelector) 
      
      #Append row to .csv file 
      # csv has the following format url,date,title,corpus
      # Remove X in order to re-run
      with open("X./drive/MyDrive/webScraping/kathimerini/corpus.csv", "a+") as file:
        data=(non_iter_str(tempUrl),non_iter_str(dateElement.text[0:10]),non_iter_str(titleElement.text),non_iter_str(corpusElement.text))
        write = csv.writer(file)
        write.writerows([data])
 


1070
[   0   11   22   32   43   54   65   76   86   97  108  119  130  140
  151  162  173  184  194  205  216  227  238  248  259  270  281  292
  302  313  324  335  346  356  367  378  389  400  410  421  432  443
  454  464  475  486  497  508  518  529  540  551  561  572  583  594
  605  615  626  637  648  659  669  680  691  702  713  723  734  745
  756  767  777  788  799  810  821  831  842  853  864  875  885  896
  907  918  929  939  950  961  972  983  993 1004 1015 1026 1037 1047
 1058 1069]
1059
[   0   11   21   32   43   53   64   75   85   96  107  118  128  139
  150  160  171  182  192  203  214  224  235  246  256  267  278  289
  299  310  321  331  342  353  363  374  385  395  406  417  427  438
  449  460  470  481  492  502  513  524  534  545  556  566  577  588
  598  609  620  631  641  652  663  673  684  695  705  716  727  737
  748  759  769  780  791  802  812  823  834  844  855  866  876  887
  898  908  919  930  940  951  962  973  983  994 1005

In [ ]:
#Used to test if .csv has the proper output
import pandas as pd
df = pd.read_csv('./drive/MyDrive/webScraping/kathimerini/corpus.csv')  
df

,url,date,title,corpus
0,https://www.ekathimerini.com/sports/237244/pao...,31.01.2019,PAOK and AEK scrape through as Reds march on,PAOK and AEK scored narrow home wins for the S...
1,https://www.ekathimerini.com/economy/237199/so...,31.01.2019,Social security dues scheme almost ready,The plan that the Labor Ministry is preparing ...
2,https://www.ekathimerini.com/opinion/cartoon/2...,31.01.2019,Cartoon,NaN
3,https://www.ekathimerini.com/news/237203/fyrom...,31.01.2019,FYROM parliament rejects request for 2nd name ...,The Parliament of the Former Yugoslav Republic...
4,https://www.ekathimerini.com/economy/237192/at...,30.01.2019,ATHEX: Marginal rise for benchmark after late ...,The gains recorded by Greek stocks during Wedn...
...,...,...,...,...
3994,https://www.ekathimerini.com/multimedia/images...,01.12.2021,Athens City Hall marks World AIDS Day,Athens City Hall was lit up on Wednesday eveni...
3995,https://www.ekathimerini.com/opinion/1172791/g...,01.12.2021,Greece and ‘Global’ Britain must meet again,"In a recent poll, Greeks were asked to identif..."
3996,https://www.ekathimerini.com/multimedia/images...,01.12.2021,Exhibition on violence against women continues...,People visit the exhibition “She’s Gone-(For A...
3997,https://www.ekathimerini.com/news/1172867/at-l...,01.12.2021,At least 30 police officers implicated in citi...,The police’s Internal Affairs Division is roun...


In [ ]:
# Store already parsed
for i in range(len(dates)):
  with open("./drive/MyDrive/webScraping/kathimerini/links/file"+dates[i]+".txt", "r") as input:
    inpt = input.readlines()  
    # Get 100 evenly spaced articles  
    indices = np.round(np.linspace(0,len(inpt)-1,100)).astype(int)

    with open('./drive/MyDrive/webScraping/kathimerini/alreadyParsed.txt',"a+") as f:
      f.write(str(dates[i])+","+" ".join(str(item) for item in indices)+"\n")